In [1]:
#!g1.1
import transformers
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")

In [ ]:
#!g1.1
Чтобы запустить атаки - нам нужно загрузить саму модель

In [12]:
#!g1.1

#!g1.1
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-32GB


In [13]:
#!g1.1   


PATH_TO_DATA = '../data/'

# данные

import pandas as pd

train_df = pd.read_csv(PATH_TO_DATA + 'train.csv', index_col='id').fillna('')  # обучающий датас
valid_df = pd.read_csv(PATH_TO_DATA + 'valid.csv', index_col='id').fillna('')  # валидационный с таргетом
test_df = pd.read_csv(PATH_TO_DATA + 'test.csv', index_col='id').fillna('')  # тестовый для каггла

In [14]:
#!g1.1

#!g1.1
train_df.shape, valid_df.shape, test_df.shape

((52057, 2), (17353, 2), (17353, 1))

In [15]:
#!g1.1


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

labels = le.fit_transform(train_df.label.values)

In [16]:
#!g1.1
labels

array([3, 3, 3, ..., 2, 3, 2])

In [17]:
#!g1.1
le.inverse_transform([3,3,3])

array(['dogs', 'dogs', 'dogs'], dtype=object)

In [ ]:
#!g1.1

# загруззим модель обученную на амазоне 

#!g1.1
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)  #стандартная модель - bert-base-uncased, 

#!g1.1
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab. используем дистилированный берт
    num_labels = 6, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()




In [23]:
#!g1.1

# загрузим веса 

#!g1.1

#загрузим модель

device = torch.device("cuda") 
#model = TheModelClass(args, *kwargs) 
model.load_state_dict(torch.load('../data/amzn.bin')) 
model.to(device)

[0;31mKernelNotResponding[0m: Kernel died unexpectedly and has been restarted. If it's not coming back, please, try restarting from the main menu.

In [2]:
#!g1.1
import textattack


In [3]:
#!g1.1
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [ ]:
#!g1.1
dataset = textattack.datasets.HuggingFaceDataset("imdb", split="test")
attack = textattack.attack_recipes.TextFoolerJin2019.build(model_wrapper)
 # Attack 20 samples with CSV logging and checkpoint saved every 5 interval
attack_args = textattack.AttackArgs(num_examples=20, log_to_csv="log.csv", checkpoint_interval=5, checkpoint_dir="checkpoints", disable_stdout=True)
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()